In [3]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import cross_val_score 
from xgboost.sklearn import XGBClassifier

In [4]:
data_train = pd.read_csv('../input/train.csv')
data_test = pd.read_csv('../input/test.csv')
print(data_train.shape, data_test.shape)
pd.merge(data_test, data_train)
data_all = pd.concat([data_train, data_test])

(891, 12) (418, 11)


In [9]:

def construct_name_title_age_map(data):
    _data = pd.DataFrame.copy(data)
    print(data)
    _data.NameTitle = data.Name.apply(lambda name: re.findall(r'.+, (\w+). .+', name)[0])
    print(_data.NameTitle.unique())
    for title in _data.NameTile.unqiue():
        
    group = _data.groupby('NameTitle')
#     print(group.get_group('Mr').index)
#     age = 
construct_name_title_age_map(data_all)


      Age        Cabin Embarked      Fare  \
0    22.0          NaN        S    7.2500   
1    38.0          C85        C   71.2833   
2    26.0          NaN        S    7.9250   
3    35.0         C123        S   53.1000   
4    35.0          NaN        S    8.0500   
5     NaN          NaN        Q    8.4583   
6    54.0          E46        S   51.8625   
7     2.0          NaN        S   21.0750   
8    27.0          NaN        S   11.1333   
9    14.0          NaN        C   30.0708   
10    4.0           G6        S   16.7000   
11   58.0         C103        S   26.5500   
12   20.0          NaN        S    8.0500   
13   39.0          NaN        S   31.2750   
14   14.0          NaN        S    7.8542   
15   55.0          NaN        S   16.0000   
16    2.0          NaN        Q   29.1250   
17    NaN          NaN        S   13.0000   
18   31.0          NaN        S   18.0000   
19    NaN          NaN        C    7.2250   
20   35.0          NaN        S   26.0000   
21   34.0 

c:\users\jferroal\.venvs\ds\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """


KeyError: 'NameTitle'

In [ ]:
def convert_sex(sex_column):
    """
    Convert Sex to 
    """
    sex_map = {'male': 0, 'female': 1}
    return sex_column.apply(lambda sex: sex_map[sex])

def convert_embarked(embarked_column):
    """
    Convert Embarked
    """
    dic = {'S': 0, 'C': 1, 'Q': 2}
    return embarked_column.apply(lambda embarked: dic.get(embarked, 0))

def construct_family_size(data):
    """
    Convert FamilySize
    """
    return data.SibSp + data.Parch

def convert_age(data):
    """
    Convert Age
    """
    data_copied = pd.DataFrame.copy(data)
#     data_copied['AgeBand'] = pd.cut(data_copied['Age'], 6)
    data_copied.Age = data_copied.Age / data_copied.FamilySize
    data_copied.loc[ data_copied['Age'] <= 14, 'Age'] = 0
    data_copied.loc[(data_copied['Age'] > 14) & (data_copied['Age'] <= 28), 'Age'] = 1
    data_copied.loc[(data_copied['Age'] > 28) & (data_copied['Age'] <= 42), 'Age'] = 2
    data_copied.loc[(data_copied['Age'] > 42) & (data_copied['Age'] <= 56), 'Age'] = 3
    data_copied.loc[(data_copied['Age'] > 56) & (data_copied['Age'] <= 70), 'Age'] = 4
    data_copied.loc[(data_copied['Age'] > 70)] = 5
    data_copied['Age'] = data_copied['Age'].astype(int)
#     data_copied = data_copied.drop(['AgeBand'], axis=1)
    return data_copied

def update_nan_age(data, target_index=5, title_index=-2):
    data_name_title_group = data.groupby('NameTitle')
    data_copied = pd.DataFrame.copy(data)
    for v in data_copied.values:
        mean_age = data_name_title_group.get_group(v[title_index]).Age.mean()
        
        if pd.isna(v[target_index]) or pd.isnull(v)[target_index]:
            data_copied.loc[data_copied.PassengerId == v[0], 'Age'] = 0 if pd.isna(mean_age) else mean_age
    return data_copied



def update_nan_cabin(data, target_index=10, pclass_index=2):
    data_copied = pd.DataFrame.copy(data)
    cabin_map = {key: index for index, key in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ')}
    data_copied.Cabin = data_copied.Cabin.apply(lambda cabin: cabin if pd.isna(cabin) else cabin_map[cabin[0]])
    pclass_group = data_copied.groupby('Pclass')
    print(pclass_group.get_group(1).Cabin.value_counts().index[0])
    for v in data_copied.values:
        if pd.isna(v[target_index]) or pd.isnull(v)[target_index]:
            data_copied.loc[data_copied.PassengerId == v[0], 'Cabin'] = pclass_group.get_group(v[pclass_index]).Cabin.value_counts().index[0]
    return data_copied



def drop_unnecessary(data):
    return data.drop(['Ticket', 'Name', 'NameTitle', 'SibSp', 'Parch', 'Embarked'], axis=1)

2.0


In [ ]:
xgb_model = XGBClassifier(
    learning_rate =0.05,
     n_estimators=235,
     max_depth=5,
     min_child_weight=0.45,
     gamma=1.7,
     subsample=0.6,
     colsample_bytree=0.825,
     objective= 'binary:logistic',
     nthread=8,
     seed=27)